# 1. Import Library

In [1]:
import numpy as np
import pandas as pd
import os
import glob
import requests
import cloudscraper
from requests.exceptions import RequestException
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import math

pd.pandas.set_option('display.max_columns',None)

#### Configure Jupyter notebook to get rid of `IOPub data rate exceeded`

In [2]:
%config

# Adjust the data rate limit and rate limit window using %config magic
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.instance().iopub_data_rate_limit = 100000000  # 100 MB/s
InteractiveShell.instance().rate_limit_window = 60.0  # 60 seconds

# Verify the settings
print(f"iopub_data_rate_limit: {InteractiveShell.instance().iopub_data_rate_limit}")
print(f"rate_limit_window: {InteractiveShell.instance().rate_limit_window}")

Available objects for config:
    AliasManager
    DisplayFormatter
    HistoryManager
    IPCompleter
    IPKernelApp
    LoggingMagics
    MagicsManager
    OSMagics
    PrefilterManager
    ScriptMagics
    StoreMagics
    ZMQInteractiveShell
iopub_data_rate_limit: 100000000
rate_limit_window: 60.0


### Earthquake dataset from USGS(United States Geological Survey)

#### Data collection begins with downloading the dataset from the USGS, which provides essential scientific information to address society’s evolving needs. As the science branch of the Department of the Interior, the USGS offers extensive data and expertise in earth, water, biology, and mapping to support decisions on environmental, resource, and public safety issues. And downloaded the dataset from this source for the years 2011 to July 2024. The website allows downloading up to 20,000 entries at a time, but since there are more than 20,000 entries per year, multiple datasets were downloaded and then concatenated to form a single comprehensive dataset.

Url:`https://earthquake.usgs.gov/earthquakes/search/#%7B%22currentfeatureid%22%3Anull%2C%22mapposition%22%3A%5B%5B86.46756%2C537.1875%5D%2C%5B86.44583%2C160.3125%5D%5D%2C%22autoUpdate%22%3A%5B%22autoUpdate%22%5D%2C%22feed%22%3A%22undefined_undefined%22%2C%22listFormat%22%3A%22default%22%2C%22restrictListToMap%22%3A%5B%5D%2C%22sort%22%3A%22newest%22%2C%22basemap%22%3A%22grayscale%22%2C%22overlays%22%3A%5B%22plates%22%5D%2C%22distanceUnit%22%3A%22km%22%2C%22timezone%22%3A%22local%22%2C%22viewModes%22%3A%5B%22settings%22%2C%22map%22%5D%2C%22event%22%3Anull%2C%22search%22%3Anull%7D`

#### Function to read CSV files using pandas. where path is the location of the file in the local storage

In [3]:
def read_csv(path):
    return pd.read_csv(path)

#### To read all CSV files from a specific location using the glob.glob method 

In [4]:
csv_files_path = glob.glob('../datasets/external/*.{}'.format('csv'))
csv_files_path

# these are the list of CSV file path   

['../datasets/external/query (11).csv',
 '../datasets/external/query (2).csv',
 '../datasets/external/query (3).csv',
 '../datasets/external/query (10).csv',
 '../datasets/external/query (8).csv',
 '../datasets/external/query (17).csv',
 '../datasets/external/query (4).csv',
 '../datasets/external/query (5).csv',
 '../datasets/external/query (16).csv',
 '../datasets/external/query (9).csv',
 '../datasets/external/query (6).csv',
 '../datasets/external/query (15).csv',
 '../datasets/external/query (14).csv',
 '../datasets/external/query (7).csv',
 '../datasets/external/query (18).csv',
 '../datasets/external/query (13).csv',
 '../datasets/external/query (12).csv',
 '../datasets/external/query (1).csv']

#### concating all the csv dataset in to one dataframe

In [5]:
raw_df = pd.concat([read_csv(file) for file in csv_files_path ], ignore_index=True, axis=0)

In [6]:
raw_df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2012-02-07T09:07:50.620Z,9.906,123.227,10.0,4.1,mb,16.0,113.0,NaN,0.58,us,usp000jemt,2014-11-07T01:46:58.918Z,"8 km SSE of Jimalalud, Philippines",earthquake,NaN,NaN,NaN,1.0,reviewed,us,us
1,2012-02-07T08:31:22.870Z,12.737,48.711,10.0,4.9,mb,59.0,94.6,NaN,1.00,us,usp000jemq,2014-11-07T01:46:58.910Z,"168 km NNW of Bosaso, Somalia",earthquake,NaN,NaN,NaN,15.0,reviewed,us,us
2,2012-02-07T08:09:10.600Z,-17.947,167.423,10.0,4.6,mb,25.0,121.4,NaN,0.79,us,usp000jemp,2014-11-07T01:46:58.908Z,"97 km WSW of Port-Vila, Vanuatu",earthquake,NaN,NaN,NaN,8.0,reviewed,us,us
3,2012-02-07T07:53:54.700Z,16.189,-94.998,68.6,4.2,md,6.0,143.3,NaN,NaN,us,usp000jemn,2014-11-07T01:46:58.907Z,"2 km SW of San Mateo del Mar, Mexico",earthquake,NaN,NaN,NaN,NaN,reviewed,unm,unm
4,2012-02-07T06:03:48.260Z,-37.581,176.846,145.2,4.2,ml,91.0,137.2,NaN,NaN,us,usp000jemj,2014-11-07T01:46:58.893Z,"40 km ENE of Maketu, New Zealand",earthquake,NaN,NaN,NaN,NaN,reviewed,wel,wel


#### Shape of Dataframe

In [7]:
raw_df.shape

(357731, 22)

In [8]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357731 entries, 0 to 357730
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   time             357731 non-null  object 
 1   latitude         357731 non-null  float64
 2   longitude        357731 non-null  float64
 3   depth            357731 non-null  float64
 4   mag              357731 non-null  float64
 5   magType          357730 non-null  object 
 6   nst              158829 non-null  float64
 7   gap              321219 non-null  float64
 8   dmin             256687 non-null  float64
 9   rms              343932 non-null  float64
 10  net              357731 non-null  object 
 11  id               357731 non-null  object 
 12  updated          357731 non-null  object 
 13  place            357727 non-null  object 
 14  type             357731 non-null  object 
 15  horizontalError  261025 non-null  float64
 16  depthError       330313 non-null  floa

#### sorting the dataframe by the time column because during concatining the files entries are altered

In [9]:
raw_df = raw_df.sort_values(by='time', ascending=False).reset_index(drop = True)
raw_df.head(2)

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2024-07-09T21:41:04.717Z,38.248400,38.106400,4.75,4.1,mwr,55.0,40.0,0.58600,0.96,us,us7000my58,2024-07-09T22:07:58.040Z,"13 km WSW of Yeşilyurt, Turkey",earthquake,3.51,5.911,0.059000,28.0,reviewed,us,us
1,2024-07-09T21:10:45.300Z,18.085167,-66.650833,20.18,2.5,md,20.0,87.0,0.05979,0.09,pr,pr71454933,2024-07-09T21:26:46.520Z,"6 km NE of Tallaboa Alta, Puerto Rico",earthquake,0.25,0.350,0.150209,10.0,reviewed,pr,pr


#### Formate time column into easily readable date and time

In [10]:
formated_datetime = pd.to_datetime(raw_df['time'])

# convert time object into date and time, it would be easy to extract dates and time as per requirement
raw_df['time'] = pd.to_datetime(formated_datetime.dt.strftime('%Y-%m-%d %H:%M:%S')) 

In [11]:
raw_df.head(2)

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2024-07-09 21:41:04,38.248400,38.106400,4.75,4.1,mwr,55.0,40.0,0.58600,0.96,us,us7000my58,2024-07-09T22:07:58.040Z,"13 km WSW of Yeşilyurt, Turkey",earthquake,3.51,5.911,0.059000,28.0,reviewed,us,us
1,2024-07-09 21:10:45,18.085167,-66.650833,20.18,2.5,md,20.0,87.0,0.05979,0.09,pr,pr71454933,2024-07-09T21:26:46.520Z,"6 km NE of Tallaboa Alta, Puerto Rico",earthquake,0.25,0.350,0.150209,10.0,reviewed,pr,pr


##  Distance between surface on Earth and Moon

### Get the `distance` feature between latitude and longitude on earth and moon by using webscrapping technology with `skyfield` API from python

In [22]:
from skyfield.api import Loader, N, E, S, W, wgs84

In [23]:
# Load the ephemeris data from the JPL DE421 file
load = Loader('~/skyfield-data')

# Create a timescale
time_scale = load.timescale()

# Load the DE421 ephemeris data set
planets = load('de421.bsp')  

# Get Earth and Moon objects
earth, moon = planets['earth'], planets['moon']  

# Function to compute altitude, azimuth, and distance
def get_distance_earth_moon(date_time, latitude, longitude):
    
    time = time_scale.utc(date_time.year, date_time.month, date_time.day, 
               date_time.hour, date_time.minute, date_time.second)
    
    if latitude >= 0:
        lat_direction = N
    else:
        lat_direction = S

    if longitude >= 0:
        lon_direction = E
    else:
        lon_direction = W

    # Create Topos object for the given latitude and longitude
    location = earth + wgs84.latlon(abs(latitude) * lat_direction, abs(longitude) * lon_direction)
    
    # Compute the astrometric position of Moon as seen from the given location on Earth
    astrometric = location.at(time).observe(moon)
    
    # Get the apparent altitude and azimuth
    alt, az, d = astrometric.apparent().altaz()

    # distance is in Astronomical and 1 Astronomical unit = 149597870.7 KM
    # Convert the distance from AU to kilometers
    AU_TO_KM = 149597870.7
    distance_km = d.au * AU_TO_KM
    
    return distance_km

# Apply the function to each row in the DataFrame
raw_df['distance'] = raw_df.apply(
    lambda row: get_distance_earth_moon(row['time'], row['latitude'], row['longitude']),
    axis=1,
    result_type='expand'
)

# Round the distance upto 3 decimal places
raw_df['distance'] = raw_df['distance'].round(decimals=3)

# raw_df.to_csv('../datasets/raw/raw_distance.csv', index=False)

## Ellipsoid height in order to get gravity on each latitude and longitude on earth as per dataset

In [14]:
# raw_df = pd.read_csv('../datasets/raw/raw_distance.csv')

### Get the `gravity acceleration (g)` feature on earth based on latitude and longitude on earth's surface by using webscrapping technology with `National Geodetic Survey [https://geodesy.noaa.gov/web_services/grav-d.shtml]` API

##### Create url with latitude and longitude to get predicted gravity
reference [https://www.unavco.org/software/geodetic-utilities/geoid-height-calculator/geoid-height-calculator.html]

##### for example: https://geodesy.noaa.gov/api/gravd/gp?lat=40.0&lon=-80.0&eht=100.0 where lat, lon and eht vary place to place on earth surface
    - lat: latitude
    - lon: longitude 
    - eht: Ellipsoid height in meters 


###### In above dataset, `lat`, `lon` is given but `eht` is missing. So, get eht first in order to get predicted gravity based on lat nad lon.

##### What is Ellipsoid ?
    - An ellipsoid is a surface that can be obtained from a sphere by deforming it by means of directional scalings, or more generally, of an affine transformation. [https://en.wikipedia.org/wiki/Ellipsoid] 

##### To get the ellipsoid height (also known as the ellipsoidal or geodetic height) based on latitude and longitude, it generally need three coordinates: latitude, longitude, and ellipsoid height. But here only  latitude and longitude, so it need an additional piece of information: the ellipsoid model and a way to calculate the height.



##### Steps to Calculate Ellipsoid Height:
    - 1. Obtain Orthometric Height (H): Use a DEM or other sources to get the elevation above sea level (orthometric height).
    - 2. Obtain Geoid Undulation (N): Use a geoid model to find the geoid undulation for given latitude and longitude.  global geoid models like EGM96, EGM2008, or regional models can be used
    - 3. Calculate Ellipsoid Height (h): The ellipsoid height ℎ is given by: 
    
        ℎ = 𝐻 + 𝑁 
        
        where,
        𝐻 is the orthometric height which is Elevation above sea level in meters and 
        𝑁  is the geoid undulation in meters (relative to the ellipsoid)


##### Terminology definitions:

- Reference Ellipsoid: Here, the reference ellipsoid refers to the datum surface derived from the WGS84 reference system.
- Geoid: The gravitational equipotential surface that approximates global mean sea level.
- Geoid Undulation: Height of the geoid above the reference ellipsoid.
- Geoid Height: Same as geoid undulation.
- Orthometric Height: Height above the geoid.

#### https://pypi.org/project/pyegt/ Api to get Ellipsoid Height inorder to fetch predicated gravity based in latitude and longitude

In [15]:
# https://geodesy.noaa.gov/api/geoid/ght?lat=33.1625&lon=-115.6375&model=13

### Using Free proxy to call the api to get Ellipsoid height also known as geoid_height

In [16]:
import threading
import queue

In [17]:
 
def get_valid_proxies(proxy_file, num_threads=10):
    q = queue.Queue()
    valid_proxies = []
    
    # Read proxies from file and add them to the queue
    with open(proxy_file, 'r') as f:
        proxies = f.read().split("\n")
        for p in proxies:
            q.put(p)
    
    # Function to check proxies
    def check_proxies():
        while not q.empty():
            proxy = q.get()
            try:
                res = requests.get("https://ipinfo.io/", proxies={"http": proxy, "https": proxy}, timeout=5)
                if res.status_code == 200:
                    # print(f"Valid proxy found: {proxy}")
                    valid_proxies.append(proxy)
            except:
                pass  # Skip invalid proxies
    
    # Start threads to check proxies
    threads = []
    for _ in range(num_threads):
        thread = threading.Thread(target=check_proxies)
        thread.start()
        threads.append(thread)
    
    # Wait for all threads to finish
    for thread in threads:
        thread.join()
    
    return valid_proxies

# Example usage
proxy_file = '../datasets/proxies/free_proxy_list.txt'
valid_proxies = get_valid_proxies(proxy_file)
# print("Valid proxies:", valid_proxies)
print(len(valid_proxies))

74


### Function to fetch Ellipsoid Height from api 

In [27]:
def get_geoid_height_url(lat, lon):
    # Construct the URL using the provided latitude and longitude values
    return f"https://geodesy.noaa.gov/api/geoid/ght?lat={lat}&lon={lon}&model=13"

def fetch_geoid_height(url, proxy):
    try:
        res = requests.get(url, proxies={"http": proxy, "https": proxy}, timeout=5, verify=False)
        res.raise_for_status()
        if res.status_code == 200:
            height_data = res.json()
            return height_data.get('geoidHeight', 0) if height_data else 0
        else:
            print(f"Request failed with status code: {res.status_code}")
            return None
    except Exception as e:
        print(f"Request for {url} failed: {e}")
        return None

def process_batch(df_batch, proxies, proxy_index):
    results = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {
            executor.submit(fetch_geoid_height, row['url'], proxies[proxy_index % len(proxies)]): i
            for i, row in df_batch.iterrows()
        }
        for future in as_completed(future_to_url):
            i = future_to_url[future]
            try:
                result = future.result()
            except Exception as exc:
                result = None
            results.append((i, result))
            proxy_index += 1
    for i, result in results:
        df_batch.at[i, 'geoid_height'] = result
    return df_batch, proxy_index

def save_progress(df, filename):
    df.to_csv(filename, index=False)

def load_progress(filename):
    return pd.read_csv(filename)

def main():
    batch_size = 1000
    proxy_index = 0
    filename = '../datasets/raw/raw_geoid_heights.csv'

    # Load or initialize DataFrame
    try:
        df = load_progress(filename)
    except FileNotFoundError:
        # Assuming raw_df is your initial DataFrame with 'latitude' and 'longitude' columns
        df = raw_df.copy()
        df['url'] = df.apply(lambda row: get_geoid_height_url(row['latitude'], row['longitude']), axis=1)
        df['geoid_height'] = None
    
    
    # Process only rows with None or 'None' values in the target column
    df_to_process = df[df['geoid_height'].isna() | (df['geoid_height'] == 'None')]

    # Process DataFrame in batches
    for start in range(0, len(df_to_process), batch_size):
        end = start + batch_size
        df_batch = df[start:end].copy()
        df_batch, proxy_index = process_batch(df_batch, valid_proxies, proxy_index)
        df.update(df_batch)
        save_progress(df, filename)
        print(f"Processed batch {start // batch_size + 1}")

    print("All batches processed.")

if __name__ == "__main__":
    main()

In [12]:
raw_gravity_df = pd.read_csv('../datasets/raw/raw_geoid_heights.csv')

In [13]:
raw_gravity_df.head(2)

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,distance,url,geoid_height
0,2024-07-09 21:41:04,38.248400,38.106400,4.75,4.1,mwr,55.0,40.0,0.58600,0.96,us,us7000my58,2024-07-09T22:07:58.040Z,"13 km WSW of Yeşilyurt, Turkey",earthquake,3.51,5.911,0.059000,28.0,reviewed,us,us,403639.392,https://geodesy.noaa.gov/api/geoid/ght?lat=38....,0.000
1,2024-07-09 21:10:45,18.085167,-66.650833,20.18,2.5,md,20.0,87.0,0.05979,0.09,pr,pr71454933,2024-07-09T21:26:46.520Z,"6 km NE of Tallaboa Alta, Puerto Rico",earthquake,0.25,0.350,0.150209,10.0,reviewed,pr,pr,395082.122,https://geodesy.noaa.gov/api/geoid/ght?lat=18....,-42.023


### Calculate Local Gravity Acceleration(g) based on Latitude, Longitude and Ellipsoid height(geoid_height)
    - https://www.isobudgets.com/how-to-calculate-local-gravity/#find-local-gravity

Gravity Formula
The most commonly used formula to calculate gravity at a given latitude and height above the ellipsoid is the WGS-84 ellipsoidal gravity formula:

𝑔
(
𝜙
,
ℎ
)
= 9.780327
(
1
+
0.0053024
sin
⁡
2
(
𝜙
)
−
0.0000058
sin
⁡
2
(
2
𝜙
)
)
−
0.000003086
⋅
ℎ
g(ϕ,h)=9.780327(1+0.0053024sin 
2
 (ϕ)−0.0000058sin 
2
 (2ϕ))−0.000003086⋅h

Where:

𝑔
(
𝜙
,
ℎ
)
g(ϕ,h) is the gravity at latitude 
𝜙 and height 
ℎ in m/s²

𝜙 is the latitude in degrees
ℎ is the height above the ellipsoid in meters



Explanation of Variables:

    - 𝜙: Latitude in degrees, which needs to be converted to radians for the trigonometric calculations.
    - ℎ: Height above the ellipsoid (geoid height) in meters.
    - sin(ϕ): The sine of the latitude.
    - sin(2ϕ): The sine of twice the latitude, used to account for variations in gravity due to latitude.
    - 9.780327: The standard gravitational acceleration at the equator in m/s².
    - 0.0053024: A coefficient that adjusts for the increase in gravity with latitude.
    - 0.0000058: A coefficient that adjusts for the decrease in gravity with latitude.
    - 0.000003086: A coefficient that adjusts for the decrease in gravity with height above the ellipsoid.

In [14]:
# Function to calculate gravity
def calculate_gravity(latitude, height):
    lat_rad = math.radians(latitude)
    
    # WGS-84 ellipsoidal gravity formula
    g = 9.780327 * (1 + 0.0053024 * (math.sin(lat_rad) ** 2) - 0.0000058 * (math.sin(2 * lat_rad) ** 2)) - 0.000003086 * height
    return round(g, 4)

raw_gravity_df['gravity'] = raw_gravity_df.apply(lambda row: calculate_gravity(row['latitude'], row['geoid_height']), axis=1)

# Save the updated dataframe to a new CSV file
raw_gravity_df.to_csv('../datasets/raw/final_raw_data.csv', index=False)

### Calculate Gravitational Forces between Earth and Moon considering Its Mass and Distance between Earth and Moon

The gravitational force between the Earth and the Moon, can be calculated by using Newton's Law of Universal Gravitation:

F = G(Me*Mm)/r2

Where,

    - F is the gravitational force between two masses
    - G is the gravitational constant (6.67430 × 10^−11m^3kg^−1s^−2).
    - Me is the mass of the Earth (5.972*10^24kg).
    - Mm is the mass of the Moon (7.348*10^22kg).
    - r is the distance between the location on the Earth and the Moon in meters

In [18]:
# Constants
G = 6.67430e-11  # Gravitational constant in m^3 kg^-1 s^-2
mass_earth = 5.972e24  # Mass of the Earth in kg
mass_moon = 7.348e22  # Mass of the Moon in kg

# Function to calculate gravitational force
def calculate_gravitational_force(distance_km):
    distance_m = distance_km * 1000
    
    if distance_m <= 0:
        return float('nan')  # Return NaN for invalid distances
    
    force = G * (mass_earth * mass_moon) / (distance_m ** 2)
    return force

# Calculate gravitational force and add as a new column
raw_gravity_df['force'] = raw_gravity_df['distance'].apply(calculate_gravitational_force)

# Save the updated dataframe to a csv file
raw_gravity_df.to_csv('../datasets/raw/final_raw_data.csv', index=False)

In [19]:
raw_gravity_df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,distance,url,geoid_height,gravity,force
0,2024-07-09 21:41:04,38.248400,38.106400,4.75,4.10,mwr,55.0,40.0,0.58600,0.96,us,us7000my58,2024-07-09T22:07:58.040Z,"13 km WSW of Yeşilyurt, Turkey",earthquake,3.51,5.911,0.059000,28.0,reviewed,us,us,403639.392,https://geodesy.noaa.gov/api/geoid/ght?lat=38....,0.000,9.8001,1.797660e+20
1,2024-07-09 21:10:45,18.085167,-66.650833,20.18,2.50,md,20.0,87.0,0.05979,0.09,pr,pr71454933,2024-07-09T21:26:46.520Z,"6 km NE of Tallaboa Alta, Puerto Rico",earthquake,0.25,0.350,0.150209,10.0,reviewed,pr,pr,395082.122,https://geodesy.noaa.gov/api/geoid/ght?lat=18....,-42.023,9.7854,1.876376e+20
2,2024-07-09 19:53:06,60.538200,-151.809200,71.10,3.60,ml,NaN,NaN,NaN,0.59,ak,ak0248s79w9m,2024-07-09T20:10:45.379Z,"28 km WSW of Salamatof, Alaska",earthquake,NaN,0.700,NaN,NaN,automatic,ak,ak,399158.905,https://geodesy.noaa.gov/api/geoid/ght?lat=60....,6.574,9.8196,1.838243e+20
3,2024-07-09 19:18:07,34.113400,86.120600,10.00,4.00,mb,19.0,97.0,6.33400,0.73,us,us7000my3y,2024-07-09T20:03:09.040Z,western Xizang,earthquake,11.57,1.954,0.156000,11.0,reviewed,us,us,404343.124,https://geodesy.noaa.gov/api/geoid/ght?lat=34....,0.000,9.7966,1.791408e+20
4,2024-07-09 19:08:52,31.499167,-115.628667,8.38,2.72,ml,9.0,108.0,0.17080,0.20,ci,ci40653951,2024-07-09T19:16:17.274Z,"92 km SSW of Alberto Oviedo Mota, B.C., MX",earthquake,0.42,2.650,0.151000,23.0,automatic,ci,ci,397055.439,https://geodesy.noaa.gov/api/geoid/ght?lat=31....,0.000,9.7944,1.857772e+20


In [20]:
raw_gravity_df.shape

(357731, 27)